# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
### lag01
- 月の売り上げラグ 1,2,3,6,12
- アイテムあたりの「週Or日」平均販売個数 1,2,3,6,12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり「週Or日」平均販売個数ラグ1,2,3,6,12'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり「週Or日」平均販売個数ラグ1,2,3,6,12，'date_cat_avg_item_cnt_lag_1',
- ジャンル　あたり[月] 平均販売個数ラグ1,2,3,6,12
- アイテムカテゴリ/ショップあたり[月] 平均販売個数ラグ1,2,3,6,12
- カテゴリ/ショップあたり[月] 平均販売個数ラグ1,2,3,6,12
- ジャンル/ショップ　あたり[月] 平均販売個数ラグ1,2,3,6,12
- ASP　月当たり
### lag02
- 価格トレンド　過去6か月
- 店舗売上トレンド
- 価格下落率
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [4]:
'''category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

sales_history = pd.read_csv('sales_history.csv')
sales_history.columns = ['Date', 'StoreId', 'ItemId', 'ItemPrice', 'SoldNum']
sales_history['Date'] = pd.to_datetime(sales_history['Date'])

test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')
'''
'''170品目のリスト'''
#test_item = test['ItemId'].unique()
#test_item = pd.DataFrame({'ItemId':test_item})

'''年月のリスト'''
#yyyymm = pd.read_csv('./csv/yyyymm.csv')

'''170品目と月番号加工済みデータ '''
#target = pd.read_csv('./csv/sales_170_date_block_asp_rev.csv')

''' ラグ01（販売個数トレンド+ASP+Revenue58特徴量'''
lag01 = pd.read_csv('./csv/matrix_lag01.csv')
'''最終データ'''
# pd.read_csv('./csv/matrix_final.csv')
#test_item
'''カテゴリ名と連結'''
#ttest = pd.merge(test,item_categories, on='ItemId')
#ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('lag01データの品目数: ',lag01['ItemId'].nunique(),'\n',
      'lag01データのカテゴリ数: ',lag01['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',lag01['SoldNum'].sum(), '\n',
     '')

lag01データの品目数:  170 
 lag01データのカテゴリ数:  26 
 売上の総数:  50595.0 
 


# Revenue の追加

In [20]:
lag01['Revenue'] = lag01['SoldNum'] * lag01['ItemPrice']

In [22]:
lag01 

,date_block_num,YYYY,MM,StoreId,ItemId,ItemPrice,SoldNum,is_holiday,ItemCategoryId,Category,...,date_shop_cat_avg_item_cnt_lag_2,date_shop_cat_avg_item_cnt_lag_3,date_shop_cat_avg_item_cnt_lag_6,date_shop_cat_avg_item_cnt_lag_12,date_shop_type_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,Revenue
0,0,2018,1,0,1000001,420.000000,6.0,0.0,100,5,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,2520.0
1,1,2018,2,0,1000001,356.666667,3.0,0.0,100,5,...,0.0000,0.0000,0.0000,0.0000,1.6670,0.0000,0.0000,0.0,0.0,1070.0
2,2,2018,3,0,1000001,420.000000,1.0,0.0,100,5,...,1.6670,0.0000,0.0000,0.0000,0.6665,1.6670,0.0000,0.0,0.0,420.0
3,3,2018,4,0,1000001,420.000000,2.0,0.0,100,5,...,0.8887,1.6670,0.0000,0.0000,0.6665,0.6665,1.6670,0.0,0.0,840.0
4,4,2018,5,0,1000001,0.000000,0.0,0.0,100,5,...,0.6665,0.8887,0.0000,0.0000,0.8667,0.6665,0.6665,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,0.1666,0.2500,0.0000,0.0000,0.0000,0.0000,1.0,0.0,0.0
67316,18,2019,7,17,3500001,0.000000,0.0,0.0,350,6,...,0.0833,0.0000,0.0000,0.0833,0.0000,0.0000,0.0000,0.0,0.0,0.0
67317,19,2019,8,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,0.0833,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0
67318,20,2019,9,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,0.0000,0.1666,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0


In [5]:
def lag_feature(df,lags,col):
    tmp = df[['date_block_num','StoreId','ItemId',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','StoreId','ItemId',col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df,shifted, on=['date_block_num','StoreId','ItemId'],how='left')
    return df

# 価格トレンド

In [6]:
item_ave =  lag01.groupby('ItemId').agg({'ItemPrice':'mean'}).reset_index()
item_ave.astype(np.float16)
item_ave = item_ave.rename(columns={'ItemPrice':'AvePrice'})

In [7]:
matrix = pd.merge(lag01,item_ave,on='ItemId',how='left')


In [8]:
matrix = matrix.rename(columns={'ItemPrice':'MonthlyAvePrice'})

In [9]:
lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix,lags,'MonthlyAvePrice')

In [10]:
matrix

,date_block_num,YYYY,MM,StoreId,ItemId,MonthlyAvePrice,SoldNum,is_holiday,ItemCategoryId,Category,...,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,AvePrice,MonthlyAvePrice_lag_1,MonthlyAvePrice_lag_2,MonthlyAvePrice_lag_3,MonthlyAvePrice_lag_4,MonthlyAvePrice_lag_5,MonthlyAvePrice_lag_6
0,0,2018,1,0,1000001,420.000000,6.0,0.0,100,5,...,0.0000,0.0,0.0,231.235269,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018,2,0,1000001,356.666667,3.0,0.0,100,5,...,0.0000,0.0,0.0,231.235269,420.000000,NaN,NaN,NaN,NaN,NaN
2,2,2018,3,0,1000001,420.000000,1.0,0.0,100,5,...,0.0000,0.0,0.0,231.235269,356.666667,420.000000,NaN,NaN,NaN,NaN
3,3,2018,4,0,1000001,420.000000,2.0,0.0,100,5,...,1.6670,0.0,0.0,231.235269,420.000000,356.666667,420.000000,NaN,NaN,NaN
4,4,2018,5,0,1000001,0.000000,0.0,0.0,100,5,...,0.6665,0.0,0.0,231.235269,420.000000,420.000000,356.666667,420.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,1.0,0.0,98.762626,0.000000,0.000000,0.000000,0.0,0.0,420.0
67316,18,2019,7,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,0.0,0.0,98.762626,0.000000,0.000000,0.000000,0.0,0.0,0.0
67317,19,2019,8,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,0.0,0.0,98.762626,0.000000,0.000000,0.000000,0.0,0.0,0.0
67318,20,2019,9,17,3500001,0.000000,0.0,0.0,350,6,...,0.0000,0.0,0.0,98.762626,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [11]:
for i in lags:
    matrix['delta_price_lag_'+str(i)] = (matrix['MonthlyAvePrice_lag_'+str(i)] -matrix[
        'AvePrice'] )/ matrix['AvePrice']

In [12]:
def select_trend(row):
    if row.first_valid_index() is None:
        return None
    else:
        return row[row.first_valid_index()]
        for i in lags:
            if row['delta_price_lag'+ str(i)]:
                return row['delta_price_lag'+str(i)]
        return 0

matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0,inplace=True)

In [13]:
matrix

,date_block_num,YYYY,MM,StoreId,ItemId,MonthlyAvePrice,SoldNum,is_holiday,ItemCategoryId,Category,...,MonthlyAvePrice_lag_4,MonthlyAvePrice_lag_5,MonthlyAvePrice_lag_6,delta_price_lag_1,delta_price_lag_2,delta_price_lag_3,delta_price_lag_4,delta_price_lag_5,delta_price_lag_6,delta_price_lag
0,0,2018,1,0,1000001,420.000000,6.0,0.0,100,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1,2018,2,0,1000001,356.666667,3.0,0.0,100,5,...,NaN,NaN,NaN,0.816332,NaN,NaN,NaN,NaN,NaN,1.0
2,2,2018,3,0,1000001,420.000000,1.0,0.0,100,5,...,NaN,NaN,NaN,0.542441,0.816332,NaN,NaN,NaN,NaN,2.0
3,3,2018,4,0,1000001,420.000000,2.0,0.0,100,5,...,NaN,NaN,NaN,0.816332,0.542441,0.816332,NaN,NaN,NaN,3.0
4,4,2018,5,0,1000001,0.000000,0.0,0.0,100,5,...,420.0,NaN,NaN,0.816332,0.816332,0.542441,0.816332,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.000000,0.0,0.0,350,6,...,0.0,0.0,420.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,3.252621,17.0
67316,18,2019,7,17,3500001,0.000000,0.0,0.0,350,6,...,0.0,0.0,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,18.0
67317,19,2019,8,17,3500001,0.000000,0.0,0.0,350,6,...,0.0,0.0,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,19.0
67318,20,2019,9,17,3500001,0.000000,0.0,0.0,350,6,...,0.0,0.0,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,20.0


In [14]:
features_to_drop = ['AvePrice','MonthlyAvePrice']

In [15]:
for i in lags:
    features_to_drop += ['MonthlyAvePrice_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(features_to_drop, axis=1,inplace=True)

In [16]:
matrix

,date_block_num,YYYY,MM,StoreId,ItemId,SoldNum,is_holiday,ItemCategoryId,Category,Type,...,date_shop_cat_avg_item_cnt_lag_2,date_shop_cat_avg_item_cnt_lag_3,date_shop_cat_avg_item_cnt_lag_6,date_shop_cat_avg_item_cnt_lag_12,date_shop_type_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag
0,0,2018,1,0,1000001,6.0,0.0,100,5,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0
1,1,2018,2,0,1000001,3.0,0.0,100,5,1,...,0.0000,0.0000,0.0000,0.0000,1.6670,0.0000,0.0000,0.0,0.0,1.0
2,2,2018,3,0,1000001,1.0,0.0,100,5,1,...,1.6670,0.0000,0.0000,0.0000,0.6665,1.6670,0.0000,0.0,0.0,2.0
3,3,2018,4,0,1000001,2.0,0.0,100,5,1,...,0.8887,1.6670,0.0000,0.0000,0.6665,0.6665,1.6670,0.0,0.0,3.0
4,4,2018,5,0,1000001,0.0,0.0,100,5,1,...,0.6665,0.8887,0.0000,0.0000,0.8667,0.6665,0.6665,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.1666,0.2500,0.0000,0.0000,0.0000,0.0000,1.0,0.0,17.0
67316,18,2019,7,17,3500001,0.0,0.0,350,6,6,...,0.0833,0.0000,0.0000,0.0833,0.0000,0.0000,0.0000,0.0,0.0,18.0
67317,19,2019,8,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0833,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,19.0
67318,20,2019,9,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.1666,0.0000,0.0000,0.0000,0.0000,0.0,0.0,20.0


# 店舗売上トレンド

In [18]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67320 entries, 0 to 67319
Data columns (total 56 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   date_block_num                         67320 non-null  int64  
 1   YYYY                                   67320 non-null  int64  
 2   MM                                     67320 non-null  int64  
 3   StoreId                                67320 non-null  int64  
 4   ItemId                                 67320 non-null  int64  
 5   SoldNum                                67320 non-null  float64
 6   is_holiday                             67320 non-null  float64
 7   ItemCategoryId                         67320 non-null  int64  
 8   Category                               67320 non-null  int64  
 9   Type                                   67320 non-null  int64  
 10  SoldNum_lag_1                          67320 non-null  float64
 11  So

In [23]:
group = lag01.groupby(['date_block_num','StoreId']).agg({'Revenue':'sum'})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group, on =['date_block_num','StoreId'],how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['StoreId']).agg({'date_shop_revenue':'mean'})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['StoreId'],how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix
                          ['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1],'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'],axis=1,inplace=True)

# 最終販売月

In [24]:
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx,row in matrix.iterrows():
    key = str(row.ItemId)+' '+str(row.StoreId)
    if key not in cache:
        if row.SoldNum!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num
        

In [25]:
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx,row in matrix.iterrows():
    key = row.ItemId
    if key not in cache:
        if row.SoldNum!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num> last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num
        

# 初販売月

In [26]:
matrix['item_shop_first_sale'] = matrix[
    'date_block_num'] - matrix.groupby(['ItemId','StoreId']
                                      )['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix[
    'date_block_num'] - matrix.groupby('ItemId')['date_block_num'].transform('min')

In [27]:
matrix

,date_block_num,YYYY,MM,StoreId,ItemId,SoldNum,is_holiday,ItemCategoryId,Category,Type,...,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag,delta_revenue_lag_1,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
0,0,2018,1,0,1000001,6.0,0.0,100,5,1,...,0.0000,0.0000,0.0,0.0,0.0,NaN,-1,-1,0,0
1,1,2018,2,0,1000001,3.0,0.0,100,5,1,...,0.0000,0.0000,0.0,0.0,1.0,0.303955,1,1,1,1
2,2,2018,3,0,1000001,1.0,0.0,100,5,1,...,1.6670,0.0000,0.0,0.0,2.0,0.284912,1,1,2,2
3,3,2018,4,0,1000001,2.0,0.0,100,5,1,...,0.6665,1.6670,0.0,0.0,3.0,0.003399,1,1,3,3
4,4,2018,5,0,1000001,0.0,0.0,100,5,1,...,0.6665,0.6665,0.0,0.0,4.0,-0.227905,1,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,1.0,0.0,17.0,-0.003078,1,-1,17,17
67316,18,2019,7,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0,0.0,18.0,0.223267,1,-1,18,18
67317,19,2019,8,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0,0.0,19.0,0.268311,1,-1,19,19
67318,20,2019,9,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0,0.0,20.0,0.225952,1,-1,20,20


In [36]:
#matrix.to_csv('./csv/target_matrix.csv')

## 最初の12か月のデータはラグの値として計算しているので削除。いいかえればTestデータでは計算されないデータセット

In [28]:
matrix = matrix[matrix.date_block_num > 11]

In [29]:
matrix

,date_block_num,YYYY,MM,StoreId,ItemId,SoldNum,is_holiday,ItemCategoryId,Category,Type,...,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag,delta_revenue_lag_1,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
12,12,2019,1,0,1000001,2.0,0.0,100,5,1,...,0.3333,0.7334,1.0000,1.6670,12.0,1.023438,1,1,12,12
13,13,2019,2,0,1000001,1.0,0.0,100,5,1,...,0.7334,0.3333,0.6665,0.6665,13.0,0.405762,1,1,13,13
14,14,2019,3,0,1000001,0.0,0.0,100,5,1,...,0.6665,0.7334,0.5330,0.6665,14.0,0.145508,1,1,14,14
15,15,2019,4,0,1000001,0.0,0.0,100,5,1,...,2.0660,0.6665,0.7334,0.8667,15.0,0.244019,1,1,15,15
16,16,2019,5,0,1000001,0.0,0.0,100,5,1,...,1.8660,2.0660,0.3333,1.0000,16.0,-0.103149,1,1,16,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,1.0000,0.0000,17.0,-0.003078,1,-1,17,17
67316,18,2019,7,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0000,0.0000,18.0,0.223267,1,-1,18,18
67317,19,2019,8,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0000,0.0000,19.0,0.268311,1,-1,19,19
67318,20,2019,9,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0000,0.0000,20.0,0.225952,1,-1,20,20


# ラグの欠損値穴埋め

In [30]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0,inplace=True)
    return df
matrix = fill_na(matrix)

In [31]:
matrix

,date_block_num,YYYY,MM,StoreId,ItemId,SoldNum,is_holiday,ItemCategoryId,Category,Type,...,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag,delta_revenue_lag_1,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
12,12,2019,1,0,1000001,2.0,0.0,100,5,1,...,0.3333,0.7334,1.0000,1.6670,12.0,1.023438,1,1,12,12
13,13,2019,2,0,1000001,1.0,0.0,100,5,1,...,0.7334,0.3333,0.6665,0.6665,13.0,0.405762,1,1,13,13
14,14,2019,3,0,1000001,0.0,0.0,100,5,1,...,0.6665,0.7334,0.5330,0.6665,14.0,0.145508,1,1,14,14
15,15,2019,4,0,1000001,0.0,0.0,100,5,1,...,2.0660,0.6665,0.7334,0.8667,15.0,0.244019,1,1,15,15
16,16,2019,5,0,1000001,0.0,0.0,100,5,1,...,1.8660,2.0660,0.3333,1.0000,16.0,-0.103149,1,1,16,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67315,17,2019,6,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,1.0000,0.0000,17.0,-0.003078,1,-1,17,17
67316,18,2019,7,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0000,0.0000,18.0,0.223267,1,-1,18,18
67317,19,2019,8,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0000,0.0000,19.0,0.268311,1,-1,19,19
67318,20,2019,9,17,3500001,0.0,0.0,350,6,6,...,0.0000,0.0000,0.0000,0.0000,20.0,0.225952,1,-1,20,20


In [32]:
matrix.to_csv('./csv/matrix_final.csv',index=False)

In [33]:
pd.read_csv('./csv/matrix_dinal.csv')

FileNotFoundError: [Errno 2] File ./csv/matrix_dinal.csv does not exist: './csv/matrix_dinal.csv'